In [1]:
from abc import ABCMeta, abstractmethod
from collections import namedtuple
import numpy as np

import plotly.graph_objects as go 
import plotly.express as px 
import plotly.io as pio
from plotly.subplots import make_subplots

from scipy.optimize import minimize, dual_annealing

In [29]:
import os
from hsir import InferSIRQ, SIRQ

In [167]:
class Law(metaclass=ABCMeta):
    @staticmethod
    @abstractmethod
    def sample(n, d):
        pass

    @staticmethod
    @abstractmethod
    def loglikely(n, d, k):
        pass
   
    @staticmethod
    def likelihood(n, d, k):
        return np.exp(loglikely(n, d, k))


class Bin(Law):
    def sample(n, d):
        return np.random.binomial(n, d)
    
    def loglikely(n, d, k):
        return k*np.log(d) + (n-k)*np.log(1-d)


class Poi(Law):
    def sample(n, d):
        return np.random.poisson(n*d)
    
    def loglikely(n, d, k):
        return k*np.log(n*d) - n*d # - np.sum(np.log(np.arange(k)+1))
    

class Gau(Law):
    def sample(n, d=1):
        return n * (1 + 0.1*np.random.randn())
    
    def loglikely(n, d, k):
        return -50 * np.log(k/n)**2 

In [168]:
def Id(x, *vargs, **kvargs):
    return x


def variation1(x):
    return np.mean(np.abs(np.diff(x)))


def variation2(x):
    return np.sqrt(np.mean(np.diff(x)**2))


def elastic_net(x, mu=1):
    return (variation1(x) + mu*variation2(x)) / (1+mu)

In [182]:
Region = namedtuple('Region', 'S I R Q')
Epidemic = namedtuple('Epidemic', 'S I R Q')


class Sample:
    def __init__(self, epidemic, ts, ms, ns, law, seed=None):
        if seed is not None:
            np.random.seed(seed)
        
        positive = np.zeros_like(ts)
        for i, (t, m, n) in enumerate(zip(ts, ms, ns)):
            positive[i] = law.sample(n, epidemic.I[t]/m)
            
        self.t = ts
        self.m = ms
        self.n = ns
        self.positive = positive
        self._law = law
        
    def __repr__(self):
        return " t: {} \n m: {} \n n: {} \n positive: {}".format(self.t, self.m, self.n, self.positive)
    
    def plot(self, fig):
        fig.add_scatter(
            x=self.t, y=self.positive / self.n * self.m, 
            mode="markers", marker_symbol=1, name="Estimated", hovertemplate="%{y}"
        )
        return fig
    

class Confirmed:
    def __init__(self, epidemic, ts, law=Poi, seed=None):
        if seed is not None:
            np.random.seed(seed)
        
        c = np.zeros_like(ts)
        for i, t in enumerate(ts):
            c[i] = law.sample(epidemic.Q[t], 1)

        self.t = ts
        self.c = c
        
    def __repr__(self):
        return " t: {} \n c: {}".format(self.t, self.c)
    
    def plot(self, fig):
        fig.add_scatter(
            x=self.t, y=self.c, 
            mode="markers", marker_symbol=2, name="Confirmed", hovertemplate="%{y}"
        )
        return fig
    
    
class Resisted:
    def __init__(self, epidemic, ts, ns, law, seed=None):
        if seed is not None:
            np.random.seed(seed)
        
        positive = np.zeros_like(ts)
        ms = np.zeros_like(ts)
        for i, (t, n) in enumerate(zip(ts, ns)):
            ms[i] = epidemic.S[t] + epidemic.I[t] + epidemic.R[t]
            positive[i] = law.sample(n, epidemic.R[t] / ms[i])
            
        self.t = ts
        self.m = ms
        self.n = ns
        self.positive = positive
        self._law = law
        
    def __repr__(self):
        return " t: {} \n m: {} \n n: {} \n positive: {}".format(self.t, self.m, self.n, self.positive)
    
    def plot(self, fig):
        fig.add_scatter(
            x=self.t, y=self.positive / self.n * self.m, 
            mode="markers", marker_symbol=3, name="Resisted", hovertemplate="%{y}"
        )
        return fig

In [238]:
class JumpProcess:
    @staticmethod
    def jump(start, amplitude, wait, horizon, seed=None):
        if seed is not None:
            np.random.seed(seed)
            
        process = np.zeros(horizon)
        t = 0
        while t < horizon:
            T = int(np.random.exponential(wait))
            process[t:t+T] = start
            start *= (1 + np.random.choice([-1, 1]) * amplitude)
            t += T
        return process

    @staticmethod
    def pulse(base, wait, high, duration, horizon, seed=None):
        if seed is not None:
            np.random.seed(seed)
            
        process = np.zeros(horizon)
        t = 0
        while t < horizon:
            T = int(np.random.exponential(wait))
            process[t:t+T] = base
            t += T
            T = int(np.random.exponential(duration))
            process[t:t+T] = high
            t += T
        return process        
    
    @staticmethod
    def plot(y, log=True):
        fig = px.scatter(y=y)
        fig.update_layout(margin=dict(b=0, l=0, r=150, t=25))
        if log:
            fig.update_yaxes(type="log")
        return fig

In [184]:
class SIRQt:
    def __init__(self, beta, gamma, theta, dt=1):
        beta = np.array(beta)
        if np.isscalar(gamma):
            gamma = gamma * np.ones_like(beta)
        else:
            gamma = np.array(gamma)
        theta = np.array(theta)
        if not beta.size == gamma.size == theta.size:
            raise Exception("Dimensions not equal. β: {}, γ: {}, θ: {}".format(beta.size, gamma.size, theta.size))
            
        self.beta = beta * dt
        self.gamma = gamma * dt
        self.theta = theta * dt
    
    def self_plot(self, ls='hv', log=False, **kvargs):
        fig = go.Figure()
        fig.add_scatter(x=np.arange(len(self.beta)), y=self.beta, line_shape=ls, name='β', hovertemplate="%{y}")
        fig.add_scatter(x=np.arange(len(self.gamma)), y=self.gamma, line_shape=ls, name='γ', hovertemplate="%{y}")
        fig.add_scatter(x=np.arange(len(self.theta)), y=self.theta, line_shape=ls, name='θ', hovertemplate="%{y}")
        fig.update_layout(margin=dict(b=0, l=0, r=150, t=25))
        if log:
            fig.update_yaxes(type="log")
        return fig
    
    def __repr__(self):
        self.self_plot().show()
        return ""

    def r0(self, control=False):
        if control:
            return self.beta / (self.gamma + self.theta)
        else:
            return self.beta / self.gamma
        
    def estimate(self, region, horizon=None):
        T = self.beta.size if horizon is None else horizon
        S = np.zeros(T+1)
        I = np.zeros(T+1)
        R = np.zeros(T+1)
        Q = np.zeros(T+1)
        S[0] = region.S
        I[0] = region.I
        R[0] = region.R
        Q[0] = region.Q
        
        for t in range(T):
            if t < self.beta.size:
                beta, gamma, theta = self.beta[t], self.gamma[t], self.theta[t]
            else:
                beta, gamma, theta = self.beta[-1], self.gamma[-1], self.theta[-1]
                
            M = S[t] + I[t] + R[t] + Q[t]
            a, b, c = beta*S[t]*I[t]/M, gamma*I[t], theta*I[t]
            S[t+1] = S[t] - a
            I[t+1] = I[t] + a - b - c
            R[t+1] = R[t] + b 
            Q[t+1] = Q[t] + c
        
        return Epidemic(S, I, R, Q)
    
    @staticmethod
    def plot(epidemic, T0=0, T=None, line=None, fig=None):
        if fig is None:
            fig = go.Figure()
            fig.update_layout(margin=dict(b=0, l=0, r=150, t=25))
        if T is None:    
            T = epidemic.S.size - 1 + T0
        fig.add_scatter(x=np.arange(T0, T+1), y=epidemic.S.astype(int), name="Susceptible", hovertemplate="%{y}")
        fig.add_scatter(x=np.arange(T0, T+1), y=epidemic.I.astype(int), name="Infectious", hovertemplate="%{y}")
        fig.add_scatter(x=np.arange(T0, T+1), y=epidemic.R.astype(int), name="Removed", hovertemplate="%{y}")
        fig.add_scatter(x=np.arange(T0, T+1), y=epidemic.Q.astype(int), name="Quarantined", hovertemplate="%{y}")
        return fig

In [185]:
T = 100
beta = JumpProcess(5, 0.3, 10, T, 0)
gamma = JumpProcess(1, 0.01, 10, T, 0)
theta = JumpProcess(2, 0.2, 20, T, 1)
dynamic = SIRQt(beta.value, gamma.value, theta.value, 0.03)
dynamic

In [186]:
city = Region(99000, 1000, 0, 0)
epidemic = dynamic.estimate(city)
fig = SIRQt.plot(epidemic)
sample = Sample(epidemic, np.arange(T//10, T, T//10), 100000*np.ones(9), 1000*np.ones(9), Bin)
sample.plot(fig)
resisted = Resisted(epidemic, np.arange(T//10, T, T//10), 1000*np.ones(9), Bin)
resisted.plot(fig)
confirmed = Confirmed(epidemic, np.arange(T//10, T, T//10))
confirmed.plot(fig)

In [187]:
def loglikely(epidemic, sample=None, resisted=None, confirmed=None, 
              law_s=Bin, law_r=Bin, law_c=Poi, weight_r=1, weight_c=1, *vargs, verbose=False, **kvargs):
    ll = 0

    if sample is not None:
        ms = sample.m
        ns = sample.n
        ds = epidemic.I[sample.t] / ms
        ks = sample.positive
        ll = sum(law_s.loglikely(n, d, k) for n, d, k in zip(ns, ds, ks)) / len(ns)
    if verbose:
        print('sample', ll)

    if resisted is not None:
        ms = resisted.m
        ns = resisted.n
        ds = epidemic.R[resisted.t] / ms
        ks = resisted.positive
        ll += weight_r * sum(law_s.loglikely(n, d, k) for n, d, k in zip(ns, ds, ks)) / len(ns)
    if verbose:
        print('resisted', ll)
        
    if confirmed is not None:
        qs = epidemic.Q[confirmed.t]
        cs = confirmed.c
        ll += weight_c * sum(law_c.loglikely(n, 1, k) for n, k in zip(qs, cs)) / len(qs)
    if verbose:
        print('confirmed', ll)

    return ll


def likelihood(epidemic, sample=None, resisted=None, confirmed=None, 
              law_s=Bin, law_r=Bin, law_c=Poi, weight_r=1, weight_c=1, *vargs, **kvargs):
    return np.exp(loglikely(epidemic, sample, resisted, confirmed, 
                            law_s, law_r, law_c, weight_r, weight_c, *vargs, **kvargs))


class InferSIRQt:
    def __init__(self, law_s=Bin, law_r=Bin, weight_r=1, law_c=Poi, weight_c=1, 
                 penalty_b=variation1, weight_b=1, penalty_t=variation1, weight_t=1, 
                 algo='map', ascdes=(Id, Id)):
        self.law_s = law_s
        self.law_r = law_r
        self.law_c = law_c
        self.weight_r = weight_r
        self.weight_c = weight_c
        self.penalty_b = penalty_b
        self.weight_b = weight_b
        self.penalty_t = penalty_t
        self.weight_t = weight_t
        self.ascdes = ascdes
        self.algo = algo
        self.dynamic = None
        self.walker = None
        self.funcval = None
        
    def self_plot(self, **options):
        return self.dynamic.self_plot(**options)
    
    def __repr__(self):
        self.self_plot().show()
        return "Function value={}".format(self.funcval)

    def fit(self, region, sample, confirmed, **options):
        if self.algo == "map":
            self.fit_beta_gamma_theta_map(region, sample, confirmed, **options)
        elif self.algo == "mcmc":
            self.fit_beta_gamma_theta_mh(region, sample, confirmed, **options)
            
    def fit_beta_gamma_theta_map(self, region, sample, confirmed, **options):        
        asc, des = self.ascdes

        def func(x):
            x = des(x)
            d = len(x) // 2 + 1
            dynamic = SIRQt(x[1:d], x[0], x[d:])
            epidemic = dynamic.estimate(region, d-1)
            ll = -loglikely(epidemic, sample, resisted, confirmed, 
                            self.law_s, self.law_r, self.law_c, self.weight_r, self.weight_c, **options)
            ll += self.weight_b * self.penalty_b(x[1:d])
            ll += self.weight_t * self.penalty_t(x[d:])
            return ll
        
        x0 = np.ones(1 + 2 * max(confirmed.t[-1], sample.t[-1]))
        d = len(x0) // 2 + 1
        x0[0] = 0.03
        x0[1:d] = 0.2
        x0[d:] = 0.04
        res = minimize(func, asc(x0), method='nelder-mead', options={'xatol': 1e-8, 'disp': True, 'maxiter': 1000000})
#        res = dual_annealing(func, np.tile([0, 1], (len(x0), 1)), maxiter=100000, seed=0)
        res.x = des(res.x)
        self.dynamic = SIRQt(res.x[1:d], res.x[0], res.x[d:])
        self.funcval = res.fun

In [160]:
b = InferSIRQ(weight_c=0.01)
b.fit(city, sample, confirmed)
b.beta, b.gamma, b.theta

/home/zheng/2019-nCoV/hsir/law.py:36: RuntimeWarning:

invalid value encountered in log



Optimization terminated successfully.
         Current function value: 2840.533189
         Iterations: 158
         Function evaluations: 279


(0.20763220081310668, 0.03049532422894547, 0.04896612114663207)

In [161]:
fig = SIRQ.plot(SIRQ(b.beta, b.gamma, b.theta).estimate(city, T))
sample.plot(fig)
resisted.plot(fig)
confirmed.plot(fig)

In [197]:
dynamic

In [201]:
a = InferSIRQt(weight_b=400, weight_t=1000, weight_r=0.1, weight_c=0.01, penalty_b=variation1, penalty_t=variation1, ascdes=(np.log, np.exp))
a.fit(city, sample, confirmed, verbose=False)
os.system('play -nq -t alsa synth 1 sine 440')
a.dynamic 

Optimization terminated successfully.
         Current function value: -2522.361010
         Iterations: 293836
         Function evaluations: 309157


In [202]:
fig = SIRQt.plot(a.dynamic.estimate(city, T))
sample.plot(fig)
resisted.plot(fig)
confirmed.plot(fig)

In [195]:
fig = SIRQt.plot(epidemic)
sample.plot(fig)
resisted.plot(fig)
confirmed.plot(fig)

In [18]:
def jump(x, t=0, state=0, cube=None, law=uniform, wait=10, *vargs, seed=None, **options):
    x = np.array(x, dtype='float64')   # caution: must specify the datatype, in-place operation below
    d = x.size
    
    if state is None:
        state = 0
        
    if cube is None:
        cube = inf_cube(d)
    cube = np.array(cube)
    
    if not incube(x, cube):
        raise Exception(MESSAGE)
        
    if seed is not None and t==0:
        np.random.seed(seed)
        
    y = law(0, **extract_options(options, 'law'))
    if state == 0:
        if cube[0, 0] < x[0] + y < cube[0, 1]:
            x[0] += y
        state += 1
    else:
        window = int(np.random.exponential(wait))
        stop = min(d - 1, state + window)
        if incube(x[state:stop+1] + y, cube[state:stop+1, :]):
            x[state:stop+1] += y
        state = stop + 1 if stop + 1 < d else 0
    return x, state         

In [24]:
def jump2(x, t=0, state=0, cube=None, law=uniform, wait=5, *vargs, seed=None, **options):
    x = np.array(x, dtype='float64')   # caution: must specify the datatype, in-place operation below
    d = x.size
    
    if state is None:
        state = 0
        
    if cube is None:
        cube = inf_cube(d)
    cube = np.array(cube)
    
    if not incube(x, cube):
        raise Exception(MESSAGE)
        
    if seed is not None and t==0:
        np.random.seed(seed)
        
    y = law(0, **extract_options(options, 'law'))
    if state == 0:
        if cube[0, 0] < x[0] + y < cube[0, 1]:
            x[0] += y
        state += 1
    else:
        window = int(np.random.exponential(wait))
        stop = min(d - 1, state + window)
        if incube(x[state] + y, cube[state:state+1, :]):
            x[state:stop+1] = x[state] + y
        state = stop + 1 if stop + 1 < d else 0
    return x, state

In [68]:
def jump3(x, t=0, state=0, cube=None, law=uniform, wait=5, *vargs, seed=None, **options):
    x = np.array(x, dtype='float64')   # caution: must specify the datatype, in-place operation below
    d = x.size
    
    if state is None:
        state = 0
        
    if cube is None:
        cube = inf_cube(d)
    cube = np.array(cube)
    
    if not incube(x, cube):
        raise Exception(MESSAGE)
        
    if seed is not None and t==0:
        np.random.seed(seed)
        
    y = law(0, **extract_options(options, 'law'))
    if state == 0:
        if cube[0, 0] < x[0] + y < cube[0, 1]:
            x[0] += y
        state += 1
    else:
        window = int(np.random.exponential(wait))
        stop = min(d - 1, state + window)
        if incube(x[d-state] + y, cube[d-state:d-state+1, :]):
            x[d-stop:d-state+1] = x[d-state] + y
        state = stop + 1 if stop + 1 < d else 0
    return x, state